In [2]:
# Step 1: 환경 설정 및 모듈 임포트
# ------------------------------------------------------------------------------
# 스크립트를 원활하게 실행하기 위해 필요한 라이브러리와 모듈을 임포트합니다.
# 프로젝트의 루트 경로를 시스템 경로에 추가하여 QuantBT 라이브러리를 찾을 수 있도록 합니다.
# ------------------------------------------------------------------------------
import sys
import os
import asyncio
from pathlib import Path
from typing import List, Dict, Any
from datetime import datetime

# Python이 QuantBT 라이브러리를 찾을 수 있도록 프로젝트 루트 경로를 설정합니다.
# 이 코드는 스크립트 실행 위치에 관계없이 안정적으로 루트 경로를 찾습니다.
try:
    current_dir = Path(os.path.dirname(os.path.realpath(__file__)))
except NameError:
    current_dir = Path.cwd()

project_root = current_dir.parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# QuantBT의 핵심 구성 요소들을 임포트합니다.
import polars as pl
from quantbt.core.interfaces.strategy import TradingStrategy
from quantbt.core.value_objects.backtest_config import BacktestConfig
from quantbt import OrderSide, OrderType
from quantbt import Order
from quantbt.ray.bayesian_parameter_optimizer import BayesianParameterOptimizer
from quantbt.ray.optimization.parameter_space import ParameterSpace

In [3]:
# Step 2: 트레이딩 전략 정의
# ------------------------------------------------------------------------------
# 최적화할 트레이딩 전략을 정의합니다.
# 이 예제에서는 두 개의 이동평균선(SMA)을 사용하는 'SimpleSMAStrategy'를 사용합니다.
#
# TradingStrategy 클래스를 상속받아 다음 두 가지 핵심 메서드를 구현해야 합니다.
# 1. _compute_indicators_for_symbol: Polars를 사용하여 벡터화된 방식으로 지표를 사전 계산합니다.
#    백테스팅 시작 전에 한 번만 실행되어 매우 빠릅니다.
# 2. generate_signals: 계산된 지표를 바탕으로 각 시간 단계(row)마다 매매 신호를 생성합니다.
# ------------------------------------------------------------------------------
class SimpleSMAStrategy(TradingStrategy):
    """
    단순 이동 평균(SMA) 교차 전략 (Ray 최적화용)
    - 매수 신호: 단기 SMA(buy_sma)가 장기 SMA(sell_sma)를 상향 돌파할 때 (여기서는 단순화를 위해 가격 > buy_sma로 구현)
    - 매도 신호: 가격이 장기 SMA(sell_sma)를 하회할 때
    """
    def __init__(self, buy_sma: int = 15, sell_sma: int = 30, position_size_pct: float = 0.8):
        # 전략의 기본 설정을 초기화합니다.
        super().__init__(
            name="SimpleSMAStrategy",
            position_size_pct=position_size_pct,  # 자산의 80%를 한 번에 투자
            max_positions=1         # 최대 1개 포지션만 보유
        )
        self.buy_sma = buy_sma
        self.sell_sma = sell_sma
        self.position_size_pct = position_size_pct

    def _compute_indicators_for_symbol(self, symbol_data: pl.DataFrame) -> pl.DataFrame:
        """
        심볼별로 이동평균 지표를 계산합니다. (Polars 벡터 연산)
        이 메서드는 백테스팅 엔진에 의해 데이터 로딩 후 단 한 번 호출됩니다.
        """
        # 시간순으로 데이터를 정렬합니다.
        data = symbol_data.sort("timestamp")

        # Polars의 내장 함수를 사용하여 빠르고 효율적으로 SMA를 계산합니다.
        buy_sma_series = self.calculate_sma(data["close"], self.buy_sma)
        sell_sma_series = self.calculate_sma(data["close"], self.sell_sma)

        # 계산된 지표를 새로운 컬럼으로 데이터프레임에 추가합니다.
        buy_sma_name = f"sma_{self.buy_sma}"
        sell_sma_name = f"sma_{self.sell_sma}"

        # 컬럼 이름이 중복되지 않도록 처리합니다.
        columns_to_add = [buy_sma_series.alias(buy_sma_name)]
        if sell_sma_name != buy_sma_name:
            columns_to_add.append(sell_sma_series.alias(sell_sma_name))

        return data.with_columns(columns_to_add)

    def generate_signals(self, current_data: Dict[str, Any]) -> List[Order]:
        """
        각 시간 단계(row)의 데이터를 기반으로 매매 신호를 생성합니다.
        이 메서드는 백테스트 루프에서 데이터 한 줄씩 호출됩니다.
        """
        orders = []
        if not self.broker:
            return orders

        # 현재 데이터 포인트에서 필요한 값들을 추출합니다.
        symbol = current_data['symbol']
        current_price = current_data['close']
        buy_sma_val = current_data.get(f'sma_{self.buy_sma}')
        sell_sma_val = current_data.get(f'sma_{self.sell_sma}', buy_sma_val)

        # 지표 값이 아직 계산되지 않은 초기 구간은 건너뜁니다.
        if buy_sma_val is None or sell_sma_val is None:
            return orders

        # 현재 포지션 상태를 확인합니다.
        has_position = self.get_current_positions().get(symbol, 0) > 0

        # 매수 신호: 현재 가격이 'buy_sma'를 넘고, 보유 포지션이 없을 때
        if current_price > buy_sma_val and not has_position:
            portfolio_value = self.get_portfolio_value()
            quantity = self.calculate_position_size(symbol, current_price, portfolio_value)
            if quantity > 0:
                orders.append(Order(symbol, OrderSide.BUY, quantity, OrderType.MARKET))

        # 매도 신호: 현재 가격이 'sell_sma' 아래로 떨어지고, 보유 포지션이 있을 때
        elif current_price < sell_sma_val and has_position:
            quantity = self.get_current_positions()[symbol]
            orders.append(Order(symbol, OrderSide.SELL, quantity, OrderType.MARKET))

        return orders

In [4]:
# Step 3: 백테스트 환경 설정 (BacktestConfig)
# --------------------------------------------------------------------------
# 백테스트에 필요한 기본 환경을 설정합니다.
# 어떤 자산을, 어느 기간 동안, 얼마의 자본금으로 테스트할지 등을 정의합니다.
# --------------------------------------------------------------------------
print("Step 3: 백테스트 환경 설정 중...")
config = BacktestConfig(
    symbols=["KRW-BTC"],                  # 최적화 대상 자산
    start_date=datetime(2024, 1, 1),    # 시작일
    end_date=datetime(2024, 12, 31),  # 종료일
    timeframe="1d",                         # 시간 프레임 (4시간 봉)
    initial_cash=10_000_000,              # 초기 자본금 (천만원)
    commission_rate=0.0,               # 수수료 
    slippage_rate=0.0,                 # 슬리피지 
)
print("✅ 백테스트 환경 설정 완료.")
print("-" * 70)

Step 3: 백테스트 환경 설정 중...
✅ 백테스트 환경 설정 완료.
----------------------------------------------------------------------


In [5]:
# Step 4: 파라미터 탐색 공간 정의 (ParameterSpace)
# --------------------------------------------------------------------------
# 최적화할 파라미터와 그 탐색 범위를 정의합니다.
# 베이지안 옵티마이저는 이 공간 내에서 최적의 파라미터 조합을 찾습니다.
# from_dict 클래스 메서드를 사용하면 편리하게 공간을 정의할 수 있습니다.
#   - (최소값, 최대값) 튜플: 정수형(Integer) 또는 실수형(Real) 파라미터
#   - [...] 리스트: 범주형(Categorical) 파라미터
# --------------------------------------------------------------------------
print("Step 4: 파라미터 탐색 공간 정의 중...")
param_config = {
    'buy_sma': (1, 1000),   # 'buy_sma'는 10에서 100 사이의 정수
    'sell_sma': (1, 2000),   # 'sell_sma'는 20에서 200 사이의 정수
    'position_size_pct': (0.5, 1),   # 'position_size_pct'는 0.1에서 0.9 사이의 실수
}
param_space = ParameterSpace.from_dict(param_config)
print("✅ 파라미터 탐색 공간 정의 완료:")
for name, dim in zip(param_space.dimension_names, param_space.dimensions):
    print(f"   - 파라미터 '{name}': {dim}")
print("-" * 70)

Step 4: 파라미터 탐색 공간 정의 중...
✅ 파라미터 탐색 공간 정의 완료:
   - 파라미터 'buy_sma': Integer(low=1, high=1000, prior='uniform', transform='identity')
   - 파라미터 'sell_sma': Integer(low=1, high=2000, prior='uniform', transform='identity')
   - 파라미터 'position_size_pct': Real(low=0.5, high=1, prior='uniform', transform='identity')
----------------------------------------------------------------------


In [6]:
# Step 5: 베이지안 옵티마이저 생성 및 실행
# --------------------------------------------------------------------------
# 모든 구성 요소를 결합하여 BayesianParameterOptimizer를 생성합니다.
# 옵티마이저는 내부적으로 Ray 클러스터를 설정하고, 여러 백테스트를 병렬로 실행하며,
# scikit-optimize를 사용해 다음 탐색할 파라미터를 지능적으로 결정합니다.
# --------------------------------------------------------------------------
print("Step 5: 베이지안 옵티마이저 생성 및 실행 준비 중...")
optimizer = BayesianParameterOptimizer(
    strategy_class=SimpleSMAStrategy,    # 최적화할 전략 클래스
    param_space=param_space,             # 정의된 파라미터 공간
    config=config,                       # 백테스트 환경 설정
    num_actors=32,                        # 병렬 실행에 사용할 CPU 코어 수 (Ray 액터 수)
    n_initial_points=32                  # 초기 랜덤 탐색 횟수 (이후부터 베이지안 최적화 시작)
)
print("✅ 옵티마이저 생성 완료.")
print("\n⏳ 최적화를 시작합니다... (진행 상황이 실시간으로 표시됩니다)")

# 최적화 실행!
# objective_metric: 최적화의 목표가 되는 지표 (예: 'sharpe_ratio')
# n_iter: 총 반복 실행 횟수
# early_stopping_patience: N회 이상 성능 개선이 없으면 조기 종료
results = await optimizer.optimize(
    objective_metric='sharpe_ratio',
    n_iter=1000,
    early_stopping_patience=5,
    early_stopping_min_delta=0.01
)


Step 5: 베이지안 옵티마이저 생성 및 실행 준비 중...
✅ 옵티마이저 생성 완료.

⏳ 최적화를 시작합니다... (진행 상황이 실시간으로 표시됩니다)
=============== Bayesian Optimization Progress ===============
진행률: 0/1000 (0.0%)
░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
--------------------------------------------------------------
📊 현재 성과:
   총 결과: 0개
   성공률: 0.0%
   평균 샤프비율: 0.0000
   평균 수익률: 0.0000
   평균 실행시간: 0.00초


2025-06-25 19:29:12,381	INFO worker.py:1908 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
처리중... 365/365: 100%|██████████| 365/365 [00:00<00:00]


=============== Bayesian Optimization Progress ===============
진행률: 32/1000 (3.2%), ETA: 1분 9초
█░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
--------------------------------------------------------------
📊 현재 성과:
   총 결과: 32개
   성공률: 100.0%
   평균 샤프비율: 0.0293
   평균 수익률: 0.0052
   평균 실행시간: 0.00초
   최고 샤프비율: 0.9381 (파라메터: {'buy_sma': np.int64(78), 'sell_sma': np.int64(157), 'position_size_pct': 0.6700105022808306})
=============== Bayesian Optimization Progress ===============
진행률: 32/1000 (3.2%), ETA: 3분 40초
█░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
--------------------------------------------------------------
📊 현재 성과:
   총 결과: 32개
   성공률: 100.0%
   평균 샤프비율: 0.0293
   평균 수익률: 0.0052
   평균 실행시간: 0.00초
   최고 샤프비율: 0.9381 (파라메터: {'buy_sma': np.int64(78), 'sell_sma': np.int64(157), 'position_size_pct': 0.6700105022808306})


처리중... 365/365: 100%|██████████| 365/365 [00:00<00:00] [repeated 32x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


=============== Bayesian Optimization Progress ===============
진행률: 64/1000 (6.4%), ETA: 2분 59초
███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
--------------------------------------------------------------
📊 현재 성과:
   총 결과: 64개
   성공률: 100.0%
   평균 샤프비율: 0.3546
   평균 수익률: 0.0698
   평균 실행시간: 0.00초
   최고 샤프비율: 2.0845 (파라메터: {'buy_sma': np.int64(221), 'sell_sma': np.int64(1), 'position_size_pct': 0.5})
=============== Bayesian Optimization Progress ===============
진행률: 64/1000 (6.4%), ETA: 4분 13초
███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
--------------------------------------------------------------
📊 현재 성과:
   총 결과: 64개
   성공률: 100.0%
   평균 샤프비율: 0.3546
   평균 수익률: 0.0698
   평균 실행시간: 0.00초
   최고 샤프비율: 2.0845 (파라메터: {'buy_sma': np.int64(221), 'sell_sma': np.int64(1), 'position_size_pct': 0.5})
=============== Bayesian Optimization Progress ===============
진행률: 64/1000 (6.4%), ETA: 5분 26초
███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
---------------------------------------

...

=============== Bayesian Optimization Progress ===============
진행률: 96/1000 (9.6%), ETA: 4분 17초
████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
--------------------------------------------------------------
📊 현재 성과:
   총 결과: 96개
   성공률: 100.0%
   평균 샤프비율: 0.7694
   평균 수익률: 0.1556
   평균 실행시간: 0.00초
   최고 샤프비율: 2.6058 (파라메터: {'buy_sma': np.int64(284), 'sell_sma': np.int64(1), 'position_size_pct': 0.7468417765190921})
=============== Bayesian Optimization Progress ===============
진행률: 96/1000 (9.6%), ETA: 5분 4초
████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
--------------------------------------------------------------
📊 현재 성과:
   총 결과: 96개
   성공률: 100.0%
   평균 샤프비율: 0.7694
   평균 수익률: 0.1556
   평균 실행시간: 0.00초
   최고 샤프비율: 2.6058 (파라메터: {'buy_sma': np.int64(284), 'sell_sma': np.int64(1), 'position_size_pct': 0.7468417765190921})
=============== Bayesian Optimization Progress ===============
진행률: 96/1000 (9.6%), ETA: 5분 51초
████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
----------

처리중... 365/365: 100%|██████████| 365/365 [00:00<00:00]


=============== Bayesian Optimization Progress ===============
진행률: 128/1000 (12.8%), ETA: 5분 56초
██████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
--------------------------------------------------------------
📊 현재 성과:
   총 결과: 128개
   성공률: 100.0%
   평균 샤프비율: 1.1231
   평균 수익률: 0.2030
   평균 실행시간: 0.00초
   최고 샤프비율: 2.8274 (파라메터: {'buy_sma': np.int64(275), 'sell_sma': np.int64(48), 'position_size_pct': 1.0})
=============== Bayesian Optimization Progress ===============
진행률: 128/1000 (12.8%), ETA: 6분 30초
██████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
--------------------------------------------------------------
📊 현재 성과:
   총 결과: 128개
   성공률: 100.0%
   평균 샤프비율: 1.1231
   평균 수익률: 0.2030
   평균 실행시간: 0.00초
   최고 샤프비율: 2.8274 (파라메터: {'buy_sma': np.int64(275), 'sell_sma': np.int64(48), 'position_size_pct': 1.0})
=============== Bayesian Optimization Progress ===============
진행률: 128/1000 (12.8%), ETA: 7분 4초
██████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
------------------------------

처리중... 365/365: 100%|██████████| 365/365 [00:00<00:00]


=============== Bayesian Optimization Final Result ===============
📊 현재 성과:
   총 결과: 160개
   성공률: 100.0%
   평균 샤프비율: 0.8616
   평균 수익률: 0.1664
   평균 실행시간: 0.00초
   최고 샤프비율: 2.8274 (파라메터: {'buy_sma': np.int64(275), 'sell_sma': np.int64(48), 'position_size_pct': 1.0})


In [8]:
# 최종 결과 요약
# optimize 메서드는 모든 시도의 결과를 리스트로 반환합니다.
# 이 중 가장 성능이 좋았던 결과를 찾아 출력합니다.
if results:
    best_result = max(results, key=lambda x: x['result'].get('sharpe_ratio', -999))
    print("🏆 최적 성능 파라미터 및 결과:")
    print(f"   - 파라미터: {best_result['params']}")
    print(f"   - 샤프 비율 (Sharpe Ratio): {best_result['result']['sharpe_ratio']:.4f}")
    print(f"   - 총 수익률 (Total Return): {best_result['result']['total_return']:.4f}")
    print(f"   - 최대 낙폭 (MDD): {best_result['result']['max_drawdown']:.4f}")
else:
    print("결과가 없습니다.")

🏆 최적 성능 파라미터 및 결과:
   - 파라미터: {'buy_sma': np.int64(275), 'sell_sma': np.int64(48), 'position_size_pct': 1.0}
   - 샤프 비율 (Sharpe Ratio): 2.8274
   - 총 수익률 (Total Return): 0.6365
   - 최대 낙폭 (MDD): 0.1112
